# Fine-tuning do Modelo BERT com AmazonTitles-1.3MM

Neste notebook, realizaremos o fine-tuning do modelo BERT (`bert-base-uncased`) usando o dataset "The AmazonTitles-1.3MM". O objetivo é treinar o modelo para que ele consiga gerar descrições de produtos com base em seus títulos.

In [ ]:
# Instalar as bibliotecas necessárias
!pip install datasets transformers

## 2. Importar as Bibliotecas

Agora, importaremos as bibliotecas necessárias para nosso trabalho.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, BertForMaskedLM, Trainer, TrainingArguments
import torch

## 3. Montar o Google Drive

Vamos montar o Google Drive para acessar os arquivos que contêm os dados de treinamento e teste.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 4. Carregar o Dataset de Treinamento

Carregaremos o dataset de treinamento (`trn.json`) utilizando a biblioteca `datasets`.

In [ ]:
jsonl_file = '/content/drive/My Drive/caminho_para_o_seu_arquivo/trn.json'  # Altere para o caminho correto
dataset = load_dataset('json', data_files=jsonl_file, split='train')

## 5. Processar o Dataset de Treinamento

Vamos processar os dados, criando prompts para o modelo com base nos títulos dos produtos e suas descrições.

In [ ]:
dados_processados = []
for example in dataset:
    title = example.get('title', '')
    content = example.get('content', '')
    prompt = f"Descreva o produto com o título '{title}'?"
    dados_processados.append({"input_text": prompt, "output_text": content})

## 6. Tokenizar os Dados de Treinamento

Utilizaremos o `AutoTokenizer` da biblioteca `transformers` para tokenizar os dados de entrada e saída.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['output_text'], padding="max_length", truncation=True, max_length=512)
    inputs['labels'] = outputs['input_ids']
    return inputs

dados_tokenizados = [tokenize_function(example) for example in dados_processados]

## 7. Criar o Dataset de Treinamento

Agora, criaremos o dataset de treinamento e dividiremos em conjuntos de treino e validação.

In [ ]:
from datasets import Dataset
hf_dataset = Dataset.from_list(dados_tokenizados)
train_test_split = hf_dataset.train_test_split(test_size=0.2)

## 8. Configurar e Treinar o Modelo

Iremos configurar o modelo BERT para o fine-tuning e definir os parâmetros de treinamento.

In [ ]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,
    num_train_epochs=3,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],
)

## 9. Treinar o Modelo

Agora vamos iniciar o treinamento do modelo.

In [ ]:
trainer.train()

## 10. Carregar o Conjunto de Teste

Em seguida, carregaremos o conjunto de teste (`tst.json`).

In [ ]:
tst_json_file = '/content/drive/My Drive/caminho_para_o_seu_arquivo/tst.json'  # Altere para o caminho correto
test_dataset = load_dataset('json', data_files=tst_json_file, split='train')

## 11. Processar o Conjunto de Teste

Processaremos o conjunto de teste da mesma forma que fizemos com o conjunto de treinamento.

In [ ]:
test_dados_processados = []
for example in test_dataset:
    title = example.get('title', '')
    prompt = f"Descreva o produto com o título '{title}'?"
    test_dados_processados.append({"input_text": prompt})

## 12. Tokenizar o Conjunto de Teste

Tokenizaremos o conjunto de teste para que possamos usá-lo na avaliação do modelo.

In [ ]:
test_dados_tokenizados = [tokenizer(example['input_text'], padding="max_length", truncation=True, max_length=512) for example in test_dados_processados]

## 13. Avaliar o Modelo

Iremos avaliar o modelo utilizando o conjunto de teste que preparamos.

In [ ]:
def gerar_respostas(model, tokenized_inputs):
    model.eval()
    with torch.no_grad():
        outputs = model(**tokenized_inputs)
    return outputs

respostas = []
for tokenized_input in test_dados_tokenizados:
    resposta = gerar_respostas(model, tokenized_input)
    respostas.append(resposta)

## 14. Exibir Algumas Respostas Geradas

Por fim, exibiremos algumas respostas geradas pelo modelo.

In [ ]:
print(respostas[:5])